In [1]:
# data exchange layer
# automatically validation type conversion


In [48]:
from typing import List, Optional
from pydantic import BaseModel, ValidationError, validator
from pydantic_settings import BaseSettings

In [40]:
# sample json data
data = [{"first_name": "Ashish", 
        "last_name": "Kumar",
        "address": {"street" : "123 Main Street",
                    "country" : "India",
                    "zipcode" : 123456},
        "favorite_colors": ["red", "blue", "green"]
        },
        {"first_name": "John",
         "last_name": "Doe",
         "address": {"street" : "456 Main Street",
                    "country" : "USA",
                    "zipcode" : 456789},
        "favorite_colors": ["red", "123"]}, # 123 is not a string
        {"first_name": "Jane",
         "last_name": "Doe",
         "address": {"street" : "789 Main Street",
                    "country" : "USA",
                    "zipcode" : "789012"}, # zipcode is string
        "favorite_colors": ["blue", "green", "yellow"]}, 
        {"first_name": "Jane",
         "last_name": '', # last_name is missing
         "address": {"street" : "789 Main Street",
                    "country" : "USA",
                    "zipcode" : "789012"}, # zipcode is string
        "favorite_colors": ["blue", "green", "yellow"]}]



In [41]:
class Address(BaseModel):
    street: str
    country: str = "India"
    zipcode: int

class Person(BaseModel):
    first_name: str
    last_name: Optional[str] 
    address: Optional[Address]
    favorite_colors: List[str]
    

In [42]:
import json 
from pprint import pprint

In [43]:
people  = [Person(**p) for p in data]
people

[Person(first_name='Ashish', last_name='Kumar', address=Address(street='123 Main Street', country='India', zipcode=123456), favorite_colors=['red', 'blue', 'green']),
 Person(first_name='John', last_name='Doe', address=Address(street='456 Main Street', country='USA', zipcode=456789), favorite_colors=['red', '123']),
 Person(first_name='Jane', last_name='Doe', address=Address(street='789 Main Street', country='USA', zipcode=789012), favorite_colors=['blue', 'green', 'yellow']),
 Person(first_name='Jane', last_name='', address=Address(street='789 Main Street', country='USA', zipcode=789012), favorite_colors=['blue', 'green', 'yellow'])]

In [44]:
for p in people:
    pprint(p)

# fixed zipcode and country

Person(first_name='Ashish', last_name='Kumar', address=Address(street='123 Main Street', country='India', zipcode=123456), favorite_colors=['red', 'blue', 'green'])
Person(first_name='John', last_name='Doe', address=Address(street='456 Main Street', country='USA', zipcode=456789), favorite_colors=['red', '123'])
Person(first_name='Jane', last_name='Doe', address=Address(street='789 Main Street', country='USA', zipcode=789012), favorite_colors=['blue', 'green', 'yellow'])
Person(first_name='Jane', last_name='', address=Address(street='789 Main Street', country='USA', zipcode=789012), favorite_colors=['blue', 'green', 'yellow'])


In [45]:
people_as_json = [p.json() for p in people]
people_as_json

C:\Users\sinhaas\AppData\Local\Temp\ipykernel_76004\1641282705.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  people_as_json = [p.json() for p in people]


['{"first_name":"Ashish","last_name":"Kumar","address":{"street":"123 Main Street","country":"India","zipcode":123456},"favorite_colors":["red","blue","green"]}',
 '{"first_name":"John","last_name":"Doe","address":{"street":"456 Main Street","country":"USA","zipcode":456789},"favorite_colors":["red","123"]}',
 '{"first_name":"Jane","last_name":"Doe","address":{"street":"789 Main Street","country":"USA","zipcode":789012},"favorite_colors":["blue","green","yellow"]}',
 '{"first_name":"Jane","last_name":"","address":{"street":"789 Main Street","country":"USA","zipcode":789012},"favorite_colors":["blue","green","yellow"]}']

In [66]:
class DbSettings(BaseSettings):
    name : str
    ip_address: str
    user : Optional[str]
    password : Optional[str]


class ServerSettings(BaseSettings):
    api_key: str
    version: int
    db_settings: Optional[DbSettings] = None

In [67]:
settings = ServerSettings(_env_file="data/server.env", _env_file_encoding="utf-8")
settings.db_settings = DbSettings(_env_file="data/db.env", _env_file_encoding="utf-8")
print(settings)

api_key='123_safjsd' version=7 db_settings=DbSettings(name='Ashish', ip_address='172.12.24.64', user='ashish', password='pass123')


In [63]:
settings.model_dump_json()
# version got converted to int

'{"api_key":"123_safjsd","version":7,"db_settings":{"name":"Ashish","ip_address":"172.12.24.64","user":"ashish","password":"pass123"}}'